### 반도체 박막 두께 알고리즘 분석 대회 By Dacon

In [3]:
import pandas as pd
import warnings
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

In [10]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [11]:
#독립변수와 종속변수를 분리합니다.
train_X = train.iloc[:,4:]  # 독립변수 -> 반사율(0~225)
train_Y = train.iloc[:,0:4] # 종족변수 -> layer 1~4 두께
test_X = test.iloc[:,1:]

In [12]:
train_X

,0,1,2,3,4,5,6,7,8,9,...,216,217,218,219,220,221,222,223,224,225
0,0.254551,0.258823,0.254659,0.252085,0.247678,0.253614,0.246511,0.259407,0.260862,0.242524,...,0.354750,0.369223,0.388184,0.408496,0.414564,0.429403,0.419225,0.443250,0.433414,0.465502
1,0.205062,0.225544,0.217758,0.202169,0.199633,0.207380,0.191318,0.195369,0.200536,0.197588,...,0.557203,0.573656,0.587998,0.612754,0.627825,0.633393,0.637706,0.625981,0.653231,0.637853
2,0.189196,0.165869,0.177655,0.156822,0.175094,0.177755,0.157582,0.158885,0.156911,0.166162,...,0.699864,0.708688,0.721982,0.713464,0.743030,0.741709,0.747743,0.746037,0.737356,0.750391
3,0.131003,0.120076,0.138975,0.117931,0.130566,0.131262,0.126962,0.134453,0.106717,0.127309,...,0.764786,0.763788,0.770017,0.787571,0.778866,0.776969,0.774712,0.801526,0.805305,0.784057
4,0.091033,0.086893,0.108125,0.080405,0.105917,0.077083,0.097895,0.086765,0.078676,0.075729,...,0.786677,0.802271,0.806557,0.799614,0.789333,0.804087,0.787763,0.794948,0.819105,0.801781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809995,0.349513,0.342661,0.326351,0.274503,0.253076,0.200509,0.159043,0.094644,0.074915,0.054888,...,0.534272,0.614797,0.696751,0.737556,0.794730,0.796240,0.833270,0.823509,0.836787,0.837526
809996,0.305650,0.295621,0.256275,0.220107,0.164272,0.116024,0.078614,0.041309,0.043083,0.037534,...,0.597710,0.678775,0.699154,0.747222,0.750222,0.778416,0.775639,0.751201,0.733458,0.702266
809997,0.256986,0.236297,0.185514,0.127390,0.098014,0.038709,0.032116,0.014756,0.064815,0.103352,...,0.665234,0.699309,0.700876,0.711074,0.722581,0.695186,0.656056,0.610674,0.509762,0.383782
809998,0.213730,0.177011,0.125791,0.080640,0.036302,0.007403,0.035937,0.043684,0.110988,0.178333,...,0.674421,0.688302,0.676385,0.636548,0.585522,0.532202,0.401940,0.259657,0.153737,0.129502


In [13]:
##### 케라스를 통해 모델 생성을 시작합니다.

from tensorflow import keras
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
import matplotlib.pyplot as plt
import numpy as np

In [14]:
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

# 콜백 정의
callback_list =[
    ReduceLROnPlateau(
        monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출
        factor=0.2,          # callback 호출시 학습률을 1/2로 줄인다.
        patience=5,         # epoch 5 동안 개선되지 않으면 실행
    ),
    ModelCheckpoint(
        filepath='my_model.{epoch:02d}-{val_loss:.4f}.hdf5',  #저장
        monitor='val_loss',
        save_best_only=True,    #가장 좋은 모델
    ),
    EarlyStopping(
        monitor='val_loss',  #epochs 10번동안 val_loss값이 개선되지 않으면 학습종료(과대적합 방지)
        patience=10,
    )
]


In [15]:
# mlp 모델 생성
# 은닉층의 depth와 유닛 node수를 적절히 조정하여 최적의 모델을 찾아야함.
# 학습 효율을 높이기 위해 배치 정규화 - dropout 대체
# 오차 역전파 과정에서 미분한 gradient가 지나치게 커지거나 소실되는 문제를 해결하기위해 가중치 초기화 함수 사용 - he_normal
# 각 층의 활성화 함수의 출력값 분포가 골고루 분포되도록 batch normalization 사용
# ELU는 ReLU의 특성을 공유하고, dead ReLU의 문제점을 해결할 수 있는 활성함수이다.
# 최적화 함수인 Optimaizer는 가장 많이 사용되고있는 adam을 사용하였다.

model = Sequential()

model.add(Dense(512, input_dim = 226, kernel_initializer='he_normal'))
model.add(BatchNormalization())
# 활성함수 
model.add(Activation('elu'))

model.add(Dense(1024, kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(512,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(512,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(256,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(128,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(units=4,kernel_initializer='he_normal', activation='linear'))
          
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               116224    
_________________________________________________________________
batch_normalization_7 (Batch (None, 512)               2048      
_________________________________________________________________
activation_7 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              525312    
_________________________________________________________________
batch_normalization_8 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_8 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)              

In [ ]:
# 모델 학습

base_model = model.fit(train_X, train_Y, epochs=1, batch_size=256, validation_split = 0.05, callbacks=callback_list)

Train on 769500 samples, validate on 40500 samples
Epoch 1/1
625664/769500 [=======================>......] - ETA: 34s - loss: 60.5490 - mae: 60.5491

In [ ]:
#예측값을 생성합니다.
pred_test = model.predict(test_X)

In [ ]:
#submission 파일을 생성합니다.
sample_sub = pd.read_csv('sample_submission.csv', index_col=0)
submission = sample_sub+pred_test
submission.to_csv('submission.csv')

In [ ]:
# 학습한 모델을 저장. 필요에따라 load하여 학습을 이어갈 수 있음
from keras.models import load_model
model.save('semicon_model.h5')